<a href="https://colab.research.google.com/github/mjcollins1047/Bellevue-DSC530/blob/master/assignment12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Mathew Collins
DSC 650
Week 12
Assignment 12

In [4]:
# Import libraries
from tensorflow import keras
from keras import layers 
from keras import backend as K 
from keras.models import Model 
import numpy as np 

In [10]:
# Create VAE Encoder Network
img_shape = (28, 28, 1)
batch_size = 16
latent_dim = 2
input_img = keras.Input(shape=img_shape)

x = layers.Conv2D(32, 3, padding='same', activation='relu')(input_img)
x = layers.Conv2D(64, 3, padding='same', activation='relu', strides=(2, 2))(x)
x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)

shape_before_flattening = K.int_shape(x)
x = layers.Flatten()(x) 
x = layers.Dense(32, activation='relu')(x)

z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

In [22]:
# Create latent-space sampling function (8.24)
def sampling(args):
  z_mean, z_log_var = args
  epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=1.)

  return z_mean + K.exp(z_log_var) * epsilon 
  
z = layers.Lambda(sampling)([z_mean, z_log_var])

In [24]:
# Create VAE decoder network, mapping latent space points to images
decoder_input = layers.Input(K.int_shape(z)[1:])

x = layers.Dense(np.prod(shape_before_flattening[1:]), activation='relu')(decoder_input)
x = layers.Reshape(shape_before_flattening[1:])(x)
x = layers.Conv2DTranspose(32, 3, padding = 'same', activation='relu', strides = (2,2))(x)
x = layers.Conv2D(1, 3, padding = 'same', activation = 'sigmoid')(x)

decoder = Model(decoder_input, x) 
z_decoded = decoder(z) 